In [1]:
import pandas as pd
import numpy as np

df_schedules = pd.read_pickle('../resources/data/preprocessed/schedules.pkl')

df_schedules

,season,day_number,backup_day_number,opponent,team_id,team,starter1,starter2,starter3,starter4,starter5,opponent_id
0,2011,7,6,UC Irvine,1228,Illinois,B. Cole,M. Davis,D. McCamey,D. Richardson,M. Tisdale,1414.0
1,2011,7,6,Seattle,1268,Maryland,A. Bowie,D. Gregory,S. Mosley,C. Tucker,J. Williams,1370.0
2,2011,7,6,Texas,1298,Navy,J. Avila,O. Avworo,R. Garcia,J. Sugars,M. Veazey,1400.0
3,2011,7,6,Rhode Island,1338,Pittsburgh,G. Brown,A. Gibbs,G. McGhee,B. Wanamaker,T. Zanna,1348.0
4,2011,7,6,Pittsburgh,1348,Rhode Island,R. Brooks,D. James,M. Jones,W. Martell,A. Richmond,1338.0
...,...,...,...,...,...,...,...,...,...,...,...,...
134122,2023,141,140,Wisconsin,1332,Oregon,K. Barthelemy,N. Bittle,Q. Guerrier,R. Soares,L. Wur,1458.0
134123,2023,141,140,Charlotte,1347,Radford,B. Antoine,J. Jeffers,S. Jules,S. Koureissi,D. Smith,1150.0
134124,2023,141,140,Eastern Kentucky,1381,Southern Utah,D. Allen,H. Butler,M. Fausett,T. Jones,J. Spurgin,1184.0
134125,2023,141,140,Oregon,1458,Wisconsin,S. Crowl,C. Essegian,C. Hepburn,M. Klesmit,T. Wahl,1332.0


In [2]:
df_reg = pd.read_csv('../resources/data/unprocessed/MRegularSeasonCompactResults.csv')

df_reg = df_reg.loc[(df_reg['Season'] >= 2011) & (df_reg['Season'] != 2020), ].reset_index(drop=True)

df_reg

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT
0,2011,7,1228,79,1414,65,H,0
1,2011,7,1268,105,1370,76,H,0
2,2011,7,1338,83,1348,75,H,0
3,2011,7,1400,83,1298,52,H,0
4,2011,9,1228,84,1405,45,H,0
...,...,...,...,...,...,...,...,...
62964,2023,132,1104,82,1401,63,N,0
62965,2023,132,1272,75,1222,65,A,0
62966,2023,132,1343,74,1463,65,H,0
62967,2023,132,1345,67,1336,65,N,0


In [3]:
df_teams = pd.read_csv('../resources/data/unprocessed/MTeams.csv')

df_teams

,TeamID,TeamName,FirstD1Season,LastD1Season
0,1101,Abilene Chr,2014,2023
1,1102,Air Force,1985,2023
2,1103,Akron,1985,2023
3,1104,Alabama,1985,2023
4,1105,Alabama A&M,2000,2023
...,...,...,...,...
372,1473,Lindenwood,2023,2023
373,1474,Queens NC,2023,2023
374,1475,Southern Indiana,2023,2023
375,1476,Stonehill,2023,2023


In [4]:
df_reg['WTeamName'] = pd.merge(
    df_reg, df_teams, how='left', 
    left_on=['WTeamID'], right_on=['TeamID']
)['TeamName']

df_reg['LTeamName'] = pd.merge(
    df_reg, df_teams, how='left', 
    left_on=['LTeamID'], right_on=['TeamID']
)['TeamName']

df_reg

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WTeamName,LTeamName
0,2011,7,1228,79,1414,65,H,0,Illinois,UC Irvine
1,2011,7,1268,105,1370,76,H,0,Maryland,Seattle
2,2011,7,1338,83,1348,75,H,0,Pittsburgh,Rhode Island
3,2011,7,1400,83,1298,52,H,0,Texas,Navy
4,2011,9,1228,84,1405,45,H,0,Illinois,Toledo
...,...,...,...,...,...,...,...,...,...,...
62964,2023,132,1104,82,1401,63,N,0,Alabama,Texas A&M
62965,2023,132,1272,75,1222,65,A,0,Memphis,Houston
62966,2023,132,1343,74,1463,65,H,0,Princeton,Yale
62967,2023,132,1345,67,1336,65,N,0,Purdue,Penn St


In [5]:
df_reg_reshape = pd.DataFrame({'season': df_reg['Season'], 
                               'day_number': df_reg['DayNum'], 
                               'team_id': df_reg['WTeamID'], 
                               'team_name': df_reg['WTeamName'], 
                               'opponent_id': df_reg['LTeamID'], 
                               'opponent_name': df_reg['LTeamName'], 
                               'result': np.repeat('win', df_reg.shape[0])})

temp =           pd.DataFrame({'season': df_reg['Season'], 
                               'day_number': df_reg['DayNum'], 
                               'team_id': df_reg['LTeamID'], 
                               'team_name': df_reg['LTeamName'], 
                               'opponent_id': df_reg['WTeamID'], 
                               'opponent_name': df_reg['WTeamName'], 
                               'result': np.repeat('loss', df_reg.shape[0])})

df_reg_reshape = pd.concat([df_reg_reshape, temp], axis=0).reset_index(drop=True)

df_reg_reshape

,season,day_number,team_id,team_name,opponent_id,opponent_name,result
0,2011,7,1228,Illinois,1414,UC Irvine,win
1,2011,7,1268,Maryland,1370,Seattle,win
2,2011,7,1338,Pittsburgh,1348,Rhode Island,win
3,2011,7,1400,Texas,1298,Navy,win
4,2011,9,1228,Illinois,1405,Toledo,win
...,...,...,...,...,...,...,...
125933,2023,132,1401,Texas A&M,1104,Alabama,loss
125934,2023,132,1222,Houston,1272,Memphis,loss
125935,2023,132,1463,Yale,1343,Princeton,loss
125936,2023,132,1336,Penn St,1345,Purdue,loss


In [6]:
temp1 = df_reg_reshape.merge(
    df_schedules, how='inner', on=['season', 'day_number', 'team_id', 'opponent_id']
).drop(columns=['backup_day_number', 'opponent', 'team'])

temp1

,season,day_number,team_id,team_name,opponent_id,opponent_name,result,starter1,starter2,starter3,starter4,starter5
0,2011,7,1228,Illinois,1414,UC Irvine,win,B. Cole,M. Davis,D. McCamey,D. Richardson,M. Tisdale
1,2011,7,1268,Maryland,1370,Seattle,win,A. Bowie,D. Gregory,S. Mosley,C. Tucker,J. Williams
2,2011,7,1338,Pittsburgh,1348,Rhode Island,win,G. Brown,A. Gibbs,G. McGhee,B. Wanamaker,T. Zanna
3,2011,7,1400,Texas,1298,Navy,win,D. Balbay,J. Hamilton,M. Hill,G. Johnson,C. Joseph
4,2011,9,1228,Illinois,1405,Toledo,win,B. Cole,M. Davis,D. McCamey,D. Richardson,M. Tisdale
...,...,...,...,...,...,...,...,...,...,...,...,...
119488,2023,132,1401,Texas A&M,1104,Alabama,loss,H. Coleman,D. Dennis,J. Marble,T. Radford,W. Taylor
119489,2023,132,1222,Houston,1272,Memphis,loss,T. Arceneaux,T. Mark,J. Roberts,J. Shead,J. Walker
119490,2023,132,1463,Yale,1343,Princeton,loss,E. Jarvis,I. Kelly,A. Mahoney,B. Mbeng,J. Poulakidas
119491,2023,132,1336,Penn St,1345,Purdue,loss,A. Funk,S. Lundy,K. Njie,J. Pickett,C. Wynter


In [7]:
temp1.columns

Index(['season', 'day_number', 'team_id', 'team_name', 'opponent_id',
       'opponent_name', 'result', 'starter1', 'starter2', 'starter3',
       'starter4', 'starter5'],
      dtype='object')

In [8]:
temp2 = df_reg_reshape.merge(
    df_schedules, how='inner', 
    left_on=['season', 'day_number', 'team_id', 'opponent_id'], 
    right_on=['season', 'backup_day_number', 'team_id', 'opponent_id'], 
    suffixes = ('', '_y')
).drop(columns=['day_number_y', 'backup_day_number', 'opponent', 'team'])

temp2

,season,day_number,team_id,team_name,opponent_id,opponent_name,result,starter1,starter2,starter3,starter4,starter5
0,2011,12,1286,Montana St,1141,C Michigan,win,C. Anderson,B. Howard,D. Piepoli,E. Rush,R. Singleton
1,2011,14,1218,Hawaii,1141,C Michigan,win,B. Amis,V. Joaquim,Z. Johnson,J. Thomas,H. Thompson
2,2011,18,1218,Hawaii,1146,Cent Arkansas,win,B. Amis,V. Joaquim,Z. Johnson,J. Thomas,H. Thompson
3,2011,23,1385,St John's,1123,Ball St,win,M. Boothe,J. Brownlee,D. Hardy,D. Kennedy,D. Polee
4,2011,33,1120,Auburn,1115,Ark Pine Bluff,win,R. Chubb,J. Langford,A. Malone,T. Neysmith,E. Ross
...,...,...,...,...,...,...,...,...,...,...,...,...
1424,2023,101,1471,UC San Diego,1218,Hawaii,loss,R. Anderson,J. Kosakowski,F. Nwaokorie,B. Pope,J. Roquemore
1425,2023,103,1218,Hawaii,1168,CS Fullerton,loss,S. Avea,N. Coleman,B. DaSilva,K. Hepa,J. McClanahan
1426,2023,115,1218,Hawaii,1415,UC Riverside,loss,S. Avea,N. Coleman,B. DaSilva,K. Hepa,J. McClanahan
1427,2023,124,1365,Santa Clara,1362,San Francisco,loss,J. Bediako,P. Braun,K. Justice,B. Podziemski,C. Stewart


In [9]:
df_reg_reshape = pd.concat(
    [temp1, temp2], ignore_index=True
).sort_values(['season', 'day_number', 'team_id'], ignore_index=True)

df_reg_reshape

,season,day_number,team_id,team_name,opponent_id,opponent_name,result,starter1,starter2,starter3,starter4,starter5
0,2011,7,1228,Illinois,1414,UC Irvine,win,B. Cole,M. Davis,D. McCamey,D. Richardson,M. Tisdale
1,2011,7,1268,Maryland,1370,Seattle,win,A. Bowie,D. Gregory,S. Mosley,C. Tucker,J. Williams
2,2011,7,1298,Navy,1400,Texas,loss,J. Avila,O. Avworo,R. Garcia,J. Sugars,M. Veazey
3,2011,7,1338,Pittsburgh,1348,Rhode Island,win,G. Brown,A. Gibbs,G. McGhee,B. Wanamaker,T. Zanna
4,2011,7,1348,Rhode Island,1338,Pittsburgh,loss,R. Brooks,D. James,M. Jones,W. Martell,A. Richmond
...,...,...,...,...,...,...,...,...,...,...,...,...
120917,2023,132,1343,Princeton,1463,Yale,win,M. Allocco,T. Evbuomwan,K. Kellman,R. Langborg,C. Pierce
120918,2023,132,1345,Purdue,1336,Penn St,win,Z. Edey,M. Gillis,F. Loyer,B. Newman,B. Smith
120919,2023,132,1401,Texas A&M,1104,Alabama,loss,H. Coleman,D. Dennis,J. Marble,T. Radford,W. Taylor
120920,2023,132,1433,VCU,1173,Dayton,win,A. Baldwin,J. Deloach,B. Johns,N. Kern,J. Nunn


In [10]:
df_reg = df_reg.merge(
    df_reg_reshape, how='inner', 
    left_on=['Season', 'DayNum', 'WTeamID', 'LTeamID'], 
    right_on=['season', 'day_number', 'team_id', 'opponent_id']
).drop(
    columns=['season', 'day_number', 'team_id', 
             'team_name', 'opponent_id', 'opponent_name', 'result']
).rename(
    columns={
        'starter1': 'WTeamStarter1', 
        'starter2': 'WTeamStarter2', 
        'starter3': 'WTeamStarter3', 
        'starter4': 'WTeamStarter4', 
        'starter5': 'WTeamStarter5'
    }
)

df_reg

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WTeamName,LTeamName,WTeamStarter1,WTeamStarter2,WTeamStarter3,WTeamStarter4,WTeamStarter5
0,2011,7,1228,79,1414,65,H,0,Illinois,UC Irvine,B. Cole,M. Davis,D. McCamey,D. Richardson,M. Tisdale
1,2011,7,1268,105,1370,76,H,0,Maryland,Seattle,A. Bowie,D. Gregory,S. Mosley,C. Tucker,J. Williams
2,2011,7,1338,83,1348,75,H,0,Pittsburgh,Rhode Island,G. Brown,A. Gibbs,G. McGhee,B. Wanamaker,T. Zanna
3,2011,7,1400,83,1298,52,H,0,Texas,Navy,D. Balbay,J. Hamilton,M. Hill,G. Johnson,C. Joseph
4,2011,9,1228,84,1405,45,H,0,Illinois,Toledo,B. Cole,M. Davis,D. McCamey,D. Richardson,M. Tisdale
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60448,2023,132,1104,82,1401,63,N,0,Alabama,Texas A&M,C. Bediako,N. Clowney,B. Miller,J. Quinerly,M. Sears
60449,2023,132,1272,75,1222,65,A,0,Memphis,Houston,K. Davis,J. Hardaway,C. Lawson,E. McCadden,D. Williams
60450,2023,132,1343,74,1463,65,H,0,Princeton,Yale,M. Allocco,T. Evbuomwan,K. Kellman,R. Langborg,C. Pierce
60451,2023,132,1345,67,1336,65,N,0,Purdue,Penn St,Z. Edey,M. Gillis,F. Loyer,B. Newman,B. Smith


In [11]:
df_reg = df_reg.merge(
    df_reg_reshape, how='inner', 
    left_on=['Season', 'DayNum', 'LTeamID', 'WTeamID'], 
    right_on=['season', 'day_number', 'team_id', 'opponent_id']
).drop(
    columns=['season', 'day_number', 'team_id', 
             'team_name', 'opponent_id', 'opponent_name', 'result']
).rename(
    columns={
        'starter1': 'LTeamStarter1', 
        'starter2': 'LTeamStarter2', 
        'starter3': 'LTeamStarter3', 
        'starter4': 'LTeamStarter4', 
        'starter5': 'LTeamStarter5'
    }
)

df_reg

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WTeamName,LTeamName,WTeamStarter1,WTeamStarter2,WTeamStarter3,WTeamStarter4,WTeamStarter5,LTeamStarter1,LTeamStarter2,LTeamStarter3,LTeamStarter4,LTeamStarter5
0,2011,7,1228,79,1414,65,H,0,Illinois,UC Irvine,B. Cole,M. Davis,D. McCamey,D. Richardson,M. Tisdale,P. Losonsky,D. Moore,P. Rembert,D. Starring,E. Wise
1,2011,7,1268,105,1370,76,H,0,Maryland,Seattle,A. Bowie,D. Gregory,S. Mosley,C. Tucker,J. Williams,A. Broussard,C. Burrell,S. Carter,G. Gilmore,G. Lever
2,2011,7,1338,83,1348,75,H,0,Pittsburgh,Rhode Island,G. Brown,A. Gibbs,G. McGhee,B. Wanamaker,T. Zanna,R. Brooks,D. James,M. Jones,W. Martell,A. Richmond
3,2011,7,1400,83,1298,52,H,0,Texas,Navy,D. Balbay,J. Hamilton,M. Hill,G. Johnson,C. Joseph,J. Avila,O. Avworo,R. Garcia,J. Sugars,M. Veazey
4,2011,9,1228,84,1405,45,H,0,Illinois,Toledo,B. Cole,M. Davis,D. McCamey,D. Richardson,M. Tisdale,J. Anyijong,D. Dear,M. Griffin,R. Holliday,Z. Leahy
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58669,2023,132,1104,82,1401,63,N,0,Alabama,Texas A&M,C. Bediako,N. Clowney,B. Miller,J. Quinerly,M. Sears,H. Coleman,D. Dennis,J. Marble,T. Radford,W. Taylor
58670,2023,132,1272,75,1222,65,A,0,Memphis,Houston,K. Davis,J. Hardaway,C. Lawson,E. McCadden,D. Williams,T. Arceneaux,T. Mark,J. Roberts,J. Shead,J. Walker
58671,2023,132,1343,74,1463,65,H,0,Princeton,Yale,M. Allocco,T. Evbuomwan,K. Kellman,R. Langborg,C. Pierce,E. Jarvis,I. Kelly,A. Mahoney,B. Mbeng,J. Poulakidas
58672,2023,132,1345,67,1336,65,N,0,Purdue,Penn St,Z. Edey,M. Gillis,F. Loyer,B. Newman,B. Smith,A. Funk,S. Lundy,K. Njie,J. Pickett,C. Wynter


In [12]:
df_reg.loc[df_reg['LTeamStarter2'].str.contains('J. Carlos'), :]

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WTeamName,LTeamName,WTeamStarter1,WTeamStarter2,WTeamStarter3,WTeamStarter4,WTeamStarter5,LTeamStarter1,LTeamStarter2,LTeamStarter3,LTeamStarter4,LTeamStarter5
54419,2023,25,1292,64,1220,54,N,0,MTSU,Hofstra,D. Dishman,E. Lawrence,T. Lenard,T. Millin,C. Weston,N. Boachie-Yiadom,J. Carlos,D. Dubar,A. Estrada,T. Thomas
54636,2023,30,1206,81,1220,77,H,1,George Mason,Hofstra,V. Bailey,D. Cooper,D. Gaines,J. Oduro,R. Polite,N. Boachie-Yiadom,J. Carlos,D. Dubar,A. Estrada,T. Thomas
54924,2023,37,1345,85,1220,66,H,0,Purdue,Hofstra,Z. Edey,C. Furst,F. Loyer,E. Morton,B. Smith,N. Boachie-Yiadom,J. Carlos,D. Dubar,A. Marshall,T. Thomas
55061,2023,41,1269,71,1220,56,N,0,Massachusetts,Hofstra,M. Cross,R. Diggins,W. Leveque,B. Martin,T. Weeks,N. Boachie-Yiadom,J. Carlos,D. Dubar,A. Marshall,T. Thomas
55295,2023,49,1378,77,1220,70,H,0,South Florida,Hofstra,J. Chaplin,R. Conwell,T. Harris,S. Hines,R. Tchewa,N. Boachie-Yiadom,J. Carlos,D. Dubar,A. Estrada,T. Thomas
55632,2023,61,1299,81,1220,79,A,0,NC A&T,Hofstra,W. Filmore,D. Horton,J. Robinson,M. Watson,K. Woods,N. Boachie-Yiadom,J. Carlos,D. Dubar,A. Estrada,T. Thomas
56358,2023,77,1406,68,1220,47,H,0,Towson,Hofstra,R. Conway,N. Russell,S. Sylla,C. Thompson,N. Timberlake,N. Boachie-Yiadom,J. Carlos,D. Dubar,A. Estrada,T. Thomas
58505,2023,126,1423,79,1220,73,N,1,UNC Wilmington,Hofstra,M. Harden-Hayes,A. Kelly,D. Newby,S. Phillips,T. White,N. Boachie-Yiadom,J. Carlos,D. Dubar,A. Estrada,T. Thomas


In [13]:
for i in range(1, 6):
    df_reg[f'WTeamStarter{i}'] = df_reg['WTeamID'].astype(str) + ' ' + df_reg[f'WTeamStarter{i}']
    df_reg[f'LTeamStarter{i}'] = df_reg['LTeamID'].astype(str) + ' ' + df_reg[f'LTeamStarter{i}']
    
df_reg

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WTeamName,LTeamName,WTeamStarter1,WTeamStarter2,WTeamStarter3,WTeamStarter4,WTeamStarter5,LTeamStarter1,LTeamStarter2,LTeamStarter3,LTeamStarter4,LTeamStarter5
0,2011,7,1228,79,1414,65,H,0,Illinois,UC Irvine,1228 B. Cole,1228 M. Davis,1228 D. McCamey,1228 D. Richardson,1228 M. Tisdale,1414 P. Losonsky,1414 D. Moore,1414 P. Rembert,1414 D. Starring,1414 E. Wise
1,2011,7,1268,105,1370,76,H,0,Maryland,Seattle,1268 A. Bowie,1268 D. Gregory,1268 S. Mosley,1268 C. Tucker,1268 J. Williams,1370 A. Broussard,1370 C. Burrell,1370 S. Carter,1370 G. Gilmore,1370 G. Lever
2,2011,7,1338,83,1348,75,H,0,Pittsburgh,Rhode Island,1338 G. Brown,1338 A. Gibbs,1338 G. McGhee,1338 B. Wanamaker,1338 T. Zanna,1348 R. Brooks,1348 D. James,1348 M. Jones,1348 W. Martell,1348 A. Richmond
3,2011,7,1400,83,1298,52,H,0,Texas,Navy,1400 D. Balbay,1400 J. Hamilton,1400 M. Hill,1400 G. Johnson,1400 C. Joseph,1298 J. Avila,1298 O. Avworo,1298 R. Garcia,1298 J. Sugars,1298 M. Veazey
4,2011,9,1228,84,1405,45,H,0,Illinois,Toledo,1228 B. Cole,1228 M. Davis,1228 D. McCamey,1228 D. Richardson,1228 M. Tisdale,1405 J. Anyijong,1405 D. Dear,1405 M. Griffin,1405 R. Holliday,1405 Z. Leahy
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58669,2023,132,1104,82,1401,63,N,0,Alabama,Texas A&M,1104 C. Bediako,1104 N. Clowney,1104 B. Miller,1104 J. Quinerly,1104 M. Sears,1401 H. Coleman,1401 D. Dennis,1401 J. Marble,1401 T. Radford,1401 W. Taylor
58670,2023,132,1272,75,1222,65,A,0,Memphis,Houston,1272 K. Davis,1272 J. Hardaway,1272 C. Lawson,1272 E. McCadden,1272 D. Williams,1222 T. Arceneaux,1222 T. Mark,1222 J. Roberts,1222 J. Shead,1222 J. Walker
58671,2023,132,1343,74,1463,65,H,0,Princeton,Yale,1343 M. Allocco,1343 T. Evbuomwan,1343 K. Kellman,1343 R. Langborg,1343 C. Pierce,1463 E. Jarvis,1463 I. Kelly,1463 A. Mahoney,1463 B. Mbeng,1463 J. Poulakidas
58672,2023,132,1345,67,1336,65,N,0,Purdue,Penn St,1345 Z. Edey,1345 M. Gillis,1345 F. Loyer,1345 B. Newman,1345 B. Smith,1336 A. Funk,1336 S. Lundy,1336 K. Njie,1336 J. Pickett,1336 C. Wynter


In [14]:
def new_rating(winner_rating, loser_rating, winner_location):
    if winner_location == 'H':
        winner_expected = 1/(1 + 10**((loser_rating - winner_rating - 40)/400))
    elif winner_location == 'A':
        winner_expected = 1/(1 + 10**((loser_rating - winner_rating + 40)/400))
    else:
        winner_expected = 1/(1 + 10**((loser_rating - winner_rating)/400))
        
    new_winner_rating = winner_rating + (20*(1 - winner_expected))
    new_loser_rating = loser_rating - (new_winner_rating - winner_rating)
    
    return (new_winner_rating, new_loser_rating)
    # return winner_expected

new_rating(1000, 1002, 'A')

(1011.2030029046381, 990.7969970953619)

In [15]:
def season_ratings_iter(ratings, data):
    # avoid overwriting issues
    ratings = ratings.copy()
    data = data.copy()
    
    for i in range(data.shape[0]):
        winning_starters = [data.iloc[i, data.columns.get_loc(f'WTeamStarter{j}')] for j in range(1, 6)]
        winning_starters_ratings = [ratings[winning_starters[j]] for j in range(5)]
        winning_team_rating = sum(winning_starters_ratings)/5
        
        losing_starters = [data.iloc[i, data.columns.get_loc(f'LTeamStarter{j}')] for j in range(1, 6)]
        losing_starters_ratings = [ratings[losing_starters[j]] for j in range(5)]
        losing_team_rating = sum(losing_starters_ratings)/5
        
        winner_location = data.iloc[i, data.columns.get_loc('WLoc')]
        
        for index, winning_starter_rating in enumerate(winning_starters_ratings):
            new_starter_rating, _ = new_rating(winning_starter_rating, losing_team_rating, winner_location)
            ratings[winning_starters[index]] = new_starter_rating
            
        for index, losing_starter_rating in enumerate(losing_starters_ratings):
            _, new_starter_rating = new_rating(winning_team_rating, losing_starter_rating, winner_location)
            ratings[losing_starters[index]] = new_starter_rating
        
    return ratings

In [16]:
def team_season_ratings_iter(ratings, data):
    # avoid overwriting issues
    ratings = ratings.copy()
    data = data.copy()
    
    for i in range(data.shape[0]):
        winner = data.iloc[i, data.columns.get_loc('WTeamID')]
        loser = data.iloc[i, data.columns.get_loc('LTeamID')]

        winner_rating = ratings[winner]
        loser_rating = ratings[loser]
        winner_location = data.iloc[i, data.columns.get_loc('WLoc')]

        new_winner_rating, new_loser_rating = new_rating(winner_rating, loser_rating, winner_location)

        ratings[winner] = new_winner_rating
        ratings[loser] = new_loser_rating
        
    return ratings

In [35]:
def get_ratings(data, season):
    
    data = data.loc[(data['Season'] == season) & 
                    (~data['WTeamStarter5'].isna()) & 
                    (~data['LTeamStarter5'].isna()), :].copy()
    
    # get team ratings to initialize player ratings
    teams = set(data['WTeamID'])
    teams.update(data['LTeamID'])
    
    ratings = dict(zip(teams, [1000 for _ in range(len(teams))]))
    new_ordinal_ratings = sorted(ratings, key=ratings.get, reverse=True)

    for i in range(10):
        old_ordinal_ratings = new_ordinal_ratings

        ratings = team_season_ratings_iter(ratings, data)
        
    # get player ratings
    players = set()
    
    for i in range(1, 6):
        players.update(data[f'WTeamStarter{i}'])
        players.update(data[f'LTeamStarter{i}'])
        
    games_played = [((data[f'WTeamStarter1'] == player) | (data[f'LTeamStarter1'] == player) | 
                     (data[f'WTeamStarter2'] == player) | (data[f'LTeamStarter2'] == player) | 
                     (data[f'WTeamStarter3'] == player) | (data[f'LTeamStarter3'] == player) | 
                     (data[f'WTeamStarter4'] == player) | (data[f'LTeamStarter4'] == player) | 
                     (data[f'WTeamStarter5'] == player) | (data[f'LTeamStarter5'] == player)).sum()
                    for player in players]
        
    players_games_played = dict(zip(players, games_played))
    
    teams_games_played = dict(zip(teams, [((data['WTeamID'] == team) | (data['LTeamID'] == team)).sum()
                                          for team in teams]))

    ratings = dict(zip(players, [(ratings[int(player_id[:4])] + 
                                  100*players_games_played[player_id]/teams_games_played[int(player_id[:4])]
                                 ) 
                                 for player_id in players]))
    
    new_ordinal_ratings = sorted(ratings, key=ratings.get, reverse=True)

    for i in range(10):
        old_ordinal_ratings = new_ordinal_ratings

        ratings = season_ratings_iter(ratings, data)

        new_ordinal_ratings = sorted(ratings, key=ratings.get, reverse=True)        

    temp = pd.DataFrame({'season': season, 
                         'player': new_ordinal_ratings, 
                         'ordinal_rating': [i+1 for i, v in enumerate(new_ordinal_ratings)]})
    
    temp['rating'] = temp['player'].map(ratings)
    
    return temp

# get_ratings(df_reg, 2023)

In [36]:
temp = get_ratings(df_reg, 2023)

temp.head(25)

,season,player,ordinal_rating,rating
0,2023,1345 M. Gillis,1,1853.824548
1,2023,1222 M. Sasser,2,1742.790936
2,2023,1104 J. Quinerly,3,1715.146273
3,2023,1104 M. Sears,4,1686.228953
4,2023,1104 C. Bediako,5,1686.228953
5,2023,1104 B. Miller,6,1686.228953
6,2023,1104 N. Clowney,7,1685.880301
7,2023,1104 N. Burnett,8,1685.404408
8,2023,1222 T. Mark,9,1677.453919
9,2023,1222 J. Shead,10,1677.453919


In [37]:
temp.tail(25)

,season,player,ordinal_rating,rating
3053,2023,1254 C. Delancy,3054,474.993374
3054,2023,1254 T. Booker,3055,472.976654
3055,2023,1453 G. Davis,3056,472.889645
3056,2023,1342 J. McMahon,3057,472.670866
3057,2023,1254 A. Fall,3058,470.342048
3058,2023,1290 E. Minton,3059,464.467525
3059,2023,1254 R. Greene,3060,464.169081
3060,2023,1115 K. Milton,3061,459.195054
3061,2023,1189 J. Bowen,3062,458.883135
3062,2023,1254 A. Washington,3063,455.270814


In [38]:
df_reg.loc[df_reg['WTeamID'] == 1222, :]

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WTeamName,LTeamName,WTeamStarter1,WTeamStarter2,WTeamStarter3,WTeamStarter4,WTeamStarter5,LTeamStarter1,LTeamStarter2,LTeamStarter3,LTeamStarter4,LTeamStarter5
30,2011,11,1222,63,1311,62,H,1,Houston,Nicholls St,1222 A. Brown,1222 M. McNeil,1222 Z. Nixon,1222 D. Thibodeaux,1222 K. VanSlyke,1311 A. Bose,1311 K. Carter,1311 K. Franklin,1311 F. Hunter,1311 B. Martin
142,2011,13,1222,88,1108,68,H,0,Houston,Alcorn St,1222 A. Brown,1222 M. McNeil,1222 Z. Nixon,1222 D. Thibodeaux,1222 K. VanSlyke,1108 C. Davenport,1108 T. Eackles,1108 K. Sanders,1108 K. Searcy,1108 M. Starks
532,2011,22,1222,78,1418,65,H,0,Houston,Louisiana,1222 A. Brown,1222 M. McNeil,1222 Z. Nixon,1222 D. Thibodeaux,1222 K. VanSlyke,1418 J. Brown,1418 T. Bureau,1418 R. Daigle,1418 D. Perez,1418 C. Wallace
980,2011,33,1222,75,1358,71,A,1,Houston,Sam Houston St,1222 A. Brown,1222 M. McNeil,1222 Z. Nixon,1222 D. Thibodeaux,1222 K. VanSlyke,1358 A. Bootle,1358 G. Clavell,1358 J. Crow,1358 L. Pevehouse,1358 M. Williams
1074,2011,35,1222,64,1305,61,H,0,Houston,Nevada,1222 A. Brown,1222 M. McNeil,1222 Z. Nixon,1222 D. Thibodeaux,1222 K. VanSlyke,1305 J. Burris,1305 D. Burton,1305 D. Hunt,1305 K. Panzer,1305 M. Story
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58108,2023,117,1222,76,1187,57,A,0,Houston,East Carolina,1222 T. Mark,1222 J. Roberts,1222 M. Sasser,1222 J. Shead,1222 J. Walker,1187 E. Ausar,1187 Q. Diboundje,1187 R. Felton,1187 B. Johnson,1187 J. Walker
58312,2023,122,1222,83,1455,66,H,0,Houston,Wichita St,1222 D. Bowser,1222 R. Chaney,1222 T. Mark,1222 M. Sasser,1222 J. Shead,1455 X. Bell,1455 J. Pierre,1455 K. Pohto,1455 C. Porter,1455 J. Rojas
58473,2023,125,1222,67,1272,65,A,0,Houston,Memphis,1222 T. Mark,1222 J. Roberts,1222 M. Sasser,1222 J. Shead,1222 J. Walker,1272 K. Davis,1272 J. Hardaway,1272 C. Lawson,1272 E. McCadden,1272 D. Williams
58619,2023,130,1222,60,1187,46,N,0,Houston,East Carolina,1222 T. Mark,1222 J. Roberts,1222 M. Sasser,1222 J. Shead,1222 J. Walker,1187 E. Ausar,1187 L. Debaut,1187 R. Felton,1187 B. Johnson,1187 J. Walker


In [39]:
temp.loc[temp['player'].str.contains('J. Clark'), :]

,season,player,ordinal_rating,rating
46,2023,1417 J. Clark,47,1593.942868
354,2023,1301 J. Clark,355,1365.890534
979,2023,1182 J. Clark,980,1153.782263


In [40]:
temp.loc[temp['player'].str.contains('1417'), :]

,season,player,ordinal_rating,rating
31,2023,1417 A. Bona,32,1613.522849
37,2023,1417 T. Campbell,38,1603.900020
38,2023,1417 J. Jaquez,39,1603.900020
43,2023,1417 A. Bailey,44,1597.937942
46,2023,1417 J. Clark,47,1593.942868
62,2023,1417 R. Stong,63,1564.961489
73,2023,1417 D. Singleton,74,1559.486329
116,2023,1417 K. Nwuba,117,1500.922947
